In [20]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import numpy as np
import os

from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.model import create_model, LeNet300, load_model
from src.harness.pruning import prune_by_percent
from src.lottery_ticket.foundations.trainer import train

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
download_data()

In [22]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()
model = create_model(0, X_train, Y_train)


Directory 'models/model_0/initial' already exists.


In [23]:
MODEL_INDEX: int = 0
# Get initial weights
dir: str = f'models/model_{MODEL_INDEX}/initial/trial0/'
layers = [
    dir + 'layer0.npy',
    dir + 'layer1.npy',
    dir + 'layer2.npy'
]

layer_weights = {f'layer{i}': np.load(layer) for i, layer in enumerate(layers)}
# Test loading a model
model: LeNet300 = load_model(MODEL_INDEX, 0, True)

# Verify all the layer weights match
for i in range(3):
    key: str = f'layer{i}'
    assert np.array_equal(model.weights[key], layer_weights[key])

In [36]:
# Test pruning
print([(key, layer.shape) for key, layer in layer_weights.items()])
percents: dict[str: float] = {key: 0.5 for key in layer_weights}
masks: dict[str, np.array] = {key: np.ones(layer.shape) for key, layer in layer_weights.items()}
new_masks: dict[str, np.array] = prune_by_percent(percents, masks, layer_weights)
for key in new_masks:
    new_mask: np.array = new_masks[key]
    old_mask: np.array = masks[key]
    assert (old_mask.sum() / 2 - new_mask.sum()) <= 1, f'Doesn\'t match for key {key}'

[('layer0', (784, 300)), ('layer1', (300, 100)), ('layer2', (100, 10))]
117599.0 117600.0
14999.0 15000.0
499.0 500.0
